# Criptografía y Computación
## Práctica 2

<div align="right"><b>Autor</b>: Jorge Gangoso Klöck
<div align="right"><b>Fecha</b>: 02/05/2022
    

### Ejercicio 1
<b>Escribe una función que determine si una secuencia de bits cumple los postulados de Golomb.</b>

Para que una secuencia cumpla los postulados de Golomb necesitamos:
1. Un contador de bits 1 y 0 que devuelva la diferencia de cantidades de cada tipo
2. Un contador de rachas que devuelva la cantidad de rachas de cada longitud
3. Un comprobador de distancia de Hamming que devuelva false si al hacer un desplazamiento circular la distancia se ve modificada


In [1]:
import numpy as np
import bitarray as ba
def toBits (mensaje):
    return ba.bitarray(mensaje).tolist()

def ContadorBits(secuencia):
    num_1 = 0
    num_0 = 0
    for i in secuencia:
        if(i == '0'):
            num_0 += 1
        elif(i == '1'):
            num_1 += 1
        else:
            return 'Error ContadorBits'
    return np.abs(num_1-num_0)

def HammingDistance(a, b):
    y = int(a, 2)^int(b,2)
    diferencias= bin(y)[2:].zfill(len(a)) 
    distancia = 0
    for i in diferencias:
        if (i == '1'):
            distancia += 1
    return distancia

def DesplazamientoCircular(seq, K):
    N = len(seq)
    seq2 = [None]*N
    for i in range (0,N):
        seq2[(i+K)%N] = seq[i]
    r = ""
    return r.join(seq2)

def HammingConstant(secuencia):
    ham1 = DesplazamientoCircular(secuencia,1)
    distancia_inicial = HammingDistance(secuencia, ham1)
    for i in range (len(secuencia)-2):
        ham1 = DesplazamientoCircular(ham1,1)
        distancia = HammingDistance(secuencia, ham1)
        if(distancia != distancia_inicial):
            return False
    return True

def PosicionEstable(secuencia):
    for i in secuencia:
        if (secuencia[0] == secuencia[-1]):
            secuencia = DesplazamientoCircular(secuencia,1)
        else:
            return secuencia
    return secuencia #si llegamos a este punto es que la secuencia es constante (p.ej: 11111)

def CuentaRachas(secuencia):
    secuencia = PosicionEstable(secuencia)
    rachas = [0]*len(secuencia) #[rachas de 1][rachas de 2][...]
    tamano_racha = 1
    for i in range (len(secuencia)-1):
        if (secuencia[i] == secuencia[i+1]):
            tamano_racha += 1
        else:
            rachas[tamano_racha-1] += 1
            tamano_racha = 1
    rachas[tamano_racha-1] += 1
    return rachas

def PostuladosDeGolomb(secuencia):
    if(ContadorBits(secuencia) > 1):
        return 'False: Bits Desiguales'
    if(HammingConstant(secuencia) == False):
        return 'False: Distancia No Constante'
    rachas = CuentaRachas(secuencia)
    for i in range (len(rachas)-1):
        if(rachas[i] != 2*rachas[i+1]):
            if(rachas[i+1] == 1 and rachas[i+2] == 0):
                return True
            return 'False: Rachas no probables'
        
    return True

In [2]:
seqcol = '1011110'
print("Cumple la secuencia ", seqcol, " los postulados de Golomb?: ", PostuladosDeGolomb(seqcol))

seqcol = '10'
print("Cumple la secuencia ", seqcol, " los postulados de Golomb?: ", PostuladosDeGolomb(seqcol))

seqcol = '01110100'
print("Cumple la secuencia ", seqcol, " los postulados de Golomb?: ", PostuladosDeGolomb(seqcol))

seqcol = '1001110'
print("Cumple la secuencia ", seqcol, " los postulados de Golomb?: ", PostuladosDeGolomb(seqcol))


Cumple la secuencia  1011110  los postulados de Golomb?:  False: Bits Desiguales
Cumple la secuencia  10  los postulados de Golomb?:  False: Rachas no probables
Cumple la secuencia  01110100  los postulados de Golomb?:  False: Distancia No Constante
Cumple la secuencia  1001110  los postulados de Golomb?:  True


### Ejercicio 2
<b>Implementa registros lineales de desplazamiento con retroalimentación (LFSR). La entrada son los coeficientes del polinomio de conexión, la semilla, y la longitud de la secuencia de salida.
Ilustra con ejemplos la dependencia del periodo de la semilla en el caso de polinomios reducibles, la independencia en el caso de polinomios irreducibles, y la maximalidad del periodo en el caso de polinomios primitivos.
Comprueba que los ejemplos con polinomios primitivos satisfacen los postulados de Golomb</b>

1. Se ha creado una función LFSR que a partir de una semilla y el polinomio de conexión genera la siguiente semilla.
2. Se ha creado también una función Periodo que dado un polinomio de conexión y una semilla calcula cual es el periodo de la secuencia generada.
3. Se ha creado finalmente una función LFSR_Stream que tal y como pide el ejercicio toma como parámetros un polinomio, una semilla y una longitud de salida y genera una secuencia de tamaño [Longitud de Salida] incluyendo la semilla original.

Mencionar brevemente que no se consideran semillas de tamaño distinto al grado del polinomio ya que si la semilla es menor no se puede operar, a menos que se consideren por defecto a un valor los valores restantes, o bien la semilla es más grande y consiste en ruido a la izquierda concatenado a una semilla real. Por tanto consideraremos por definición una semilla como un vector de tamaño L necesariamente.

Se deja una celda de código bajo el código de prueba para realizar las comprobaciones necesarias. Las semillas están formateadas como [s0, s1, ..., sL-1], y los polinomios como [a,b,...,z] donde a,b,...,z son los exponentes en los que el polinomio tiene coeficiente 1.

In [3]:
import numpy as np
def LFSR (polinomio, seed):
    longitud = len(seed)
    polinomio = [longitud-i for i in polinomio]
    M = np.zeros((longitud, longitud),int)
    for i in range (0, longitud-1):
        M[i,i+1]=1
    M[-1,polinomio]=1
    return list(np.matmul(M,seed)%2)

# def Periodo (polinomio, ini_seed):
#     count = 0
#     seed = ini_seed
#     longitud = len(seed)
#     for i in range (0, 2**longitud):
#         count += 1
#         seed = LFSR(polinomio, seed)
#         if (seed == ini_seed):
#             break
#     return count

def Periodo (polinomio, ini_seed):
    seeds = []
    seeds.append(ini_seed.copy())
    seed = ini_seed.copy()
    for i in range (0, 2**len(ini_seed)):
        seed = LFSR(polinomio, seed)
        if (seed in seeds):
            return i+1
        else:
            seeds.append(seed.copy())
    return i+1

def LFSR_Stream (polinomio, ini_seed, exit_len):
    seed = ini_seed
    stream = ini_seed
    for i in range (0, exit_len-len(ini_seed)):
        seed = LFSR(polinomio, seed)
        stream.append(seed[-1])
    return stream
    

In [4]:
print("**Estudio del Periodo en el caso de polinomio reducible en Z2, el valor máximo como siempre es 2^grado-1")
print("El valor esperado es menor al máximo y variable según la semilla elegida**\n")
polinomio = [2, 4] #1+ x^2 + x^4 Reducible

seed = [1,1,0,0]
print("Periodo máximo: ", (2**4)-1)
print("Periodo con seed 1: ", Periodo(polinomio, seed))
stream = LFSR_Stream(polinomio, seed, (2**4)-1)
#print(stream)

seed = [1,1,0,1]
print("Periodo con seed 2: ", Periodo(polinomio, seed))
stream = LFSR_Stream(polinomio, seed, (2**4)-1)
#print(stream)

print("\n**Estudio del Periodo en el caso de polinomio primitivo en Z2, el valor máximo como siempre es 2^grado-1")
print("El valor esperado el máximo e independiente de la semilla elegida")
print("Además, un flujo de bits de tamaño 2^grado-1 cumplirá los postulados de Golomb**\n")

polinomio = [1, 4] #1 + x + x^4 Primitivo 

seed = [1,0,0,1]
print("Periodo máximo: ", (2**4)-1)
print("Periodo con seed 1: ", Periodo(polinomio, seed))
stream = LFSR_Stream(polinomio, seed, (2**4)-1)
#print(stream)

seed = [1,1,1,0]
print("Periodo con seed 2: ", Periodo(polinomio, seed))
stream = LFSR_Stream(polinomio, seed, (2**4)-1)
print("Secuencia: ", stream)
stream = ''.join(str(i) for i in stream)
print("Cumple los postulados de Golomb: ", PostuladosDeGolomb(stream))

print("\n**Estudio del Periodo en el caso de polinomio irreducible (pero no primitivo) en Z2, el valor máximo como siempre es 2^grado-1")
print("El valor esperado es divisor del máximo pero constante independiente de la semilla elegida**\n")

polinomio = [1,2,3,4] # 1 + x + x^2 + x^3 + x^4 Irreducible, No primitivo

seed = [1,0,1,0]
print("Periodo máximo: ", (2**4)-1)
print("Periodo con seed 1: ", Periodo(polinomio, seed))
stream = LFSR_Stream(polinomio, seed, (2**4)-1)
#print(stream)

seed = [0,1,0,0]
print("Periodo con seed 2: ", Periodo(polinomio, seed))
stream = LFSR_Stream(polinomio, seed, (2**4)-1)
#print(stream)

**Estudio del Periodo en el caso de polinomio reducible en Z2, el valor máximo como siempre es 2^grado-1
El valor esperado es menor al máximo y variable según la semilla elegida**

Periodo máximo:  15
Periodo con seed 1:  6
Periodo con seed 2:  3

**Estudio del Periodo en el caso de polinomio primitivo en Z2, el valor máximo como siempre es 2^grado-1
El valor esperado el máximo e independiente de la semilla elegida
Además, un flujo de bits de tamaño 2^grado-1 cumplirá los postulados de Golomb**

Periodo máximo:  15
Periodo con seed 1:  15
Periodo con seed 2:  15
Secuencia:  [1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1]
Cumple los postulados de Golomb:  True

**Estudio del Periodo en el caso de polinomio irreducible (pero no primitivo) en Z2, el valor máximo como siempre es 2^grado-1
El valor esperado es divisor del máximo pero constante independiente de la semilla elegida**

Periodo máximo:  15
Periodo con seed 1:  5
Periodo con seed 2:  5


### Ejercicio 3
<b>Escribe una función que toma como argumentos una función polinómica f , una semilla s y un entero positivo k,
y devuelve una secuencia de longitud k generada al aplicar a s el registro no lineal de desplazamiento 
con retroalimentación asociado a f.  
    
Encuentra el periodo de la NLFSR ( (x ∧ y)∨ ¬z )⊕ t con semilla 1011.</b>

In [5]:
def NLFSR (funcion, seed):
    results = []
    for i in range (0, len(seed)):
        results.append(int(all([seed[j]**funcion[i][j] for j in range (0, len(seed))])))
    next = sum(results)%2
    seed.append(next)
    seed.pop(0)
    return seed

def NLFSR_Stream (funcion, ini_seed, k):
    seed = ini_seed
    stream = ini_seed.copy()
    for i in range (0, k-len(ini_seed)):
        seed = NLFSR(funcion, seed) 
        stream.append(seed[-1])
    return stream

def Periodo_NLFSR (funcion, ini_seed):
    seeds = []
    seeds.append(ini_seed.copy())
    seed = ini_seed.copy()
    for i in range (0, 2**len(ini_seed)):
        seed = NLFSR(funcion, seed)
        if (seed in seeds):
            return i+1
        else:
            seeds.append(seed.copy())
    return i+1
        

In [6]:
f_ejercicio = [[0,0,0,0],[0,0,1,0],[1,1,1,0],[0,0,0,1]] #Polinomio de Gigalkine de la función

seed = [1,0,1,1]
print(NLFSR_Stream(f_ejercicio, seed, 2**len(seed)))

seed = [1,0,1,1]
print(Periodo_NLFSR(f_ejercicio, seed))


[1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]
9


### Ejercicio 4
<b>Implementa el generador de Geffe.  
    
Encuentra ejemplos donde el periodo de la salida es p1p2p3, con p1, p2 y p3 los periodos de los tres
LFSRs usados en el generador de Geffe.  
    
Usa este ejercicio para construir un cifrado en flujo. Con entrada un mensaje m, construye una llave k
con la misma longitud que m, y devuelve m⊕k (donde ⊕ significa suma componente a componente en
Z2).</b>

Los polinomios primitivos son escogidos de antemano para asegurar que sus periodos son primos relativos entre sí, se podrían parametrizar pero entonces quedaría a merced del usuario el asegurarse de que introduce polinomios que cumplan las restricciones. Las semillas se inicializan de la misma manera, para evitar el engorro de especificar de qué tamaño debe ser cada semilla se crean dentro de la propia función Geffe. Modificar esto no complicaría el ejercicio pero al no estar especificado queda un poco en el aire la forma correcta de operar en este caso con los parámetros.

In [7]:
def FuncionGeffe(l1, l2, l3):
    result = [None]*len(l1)
    for i in range (0, len(l1)):
        l1[i] = l1[i]*l2[i]
        l2[i] = ((1+l2[i])%2)*l3[i]
        result[i] = l1[i]^l2[i]
    return result

def Geffe (tamano_llave):
    tam = tamano_llave
    polinomio1 = [1, 4] #1 + x + x^4 Primitivo 
    seed1 = [1,0,1,1]
    polinomio2 = [2, 5] #1 + x^2 + x^5
    seed2 = [1,0,1,1,0]
    polinomio3 = [1, 7] #1 + x + x^7
    seed3 = [0,1,1,0,1,1,1]
    L1 = LFSR_Stream(polinomio1, seed1, tam)
    L2 = LFSR_Stream(polinomio2, seed2, tam)
    L3 = LFSR_Stream(polinomio3, seed3, tam)
    salida = FuncionGeffe(L1,L2,L3)
    #salida = (L1*L2)^(1+L2)*L3
    return salida

def CifradoFlujo (mensaje):
    tamano_llave = len(mensaje)
    llave = Geffe(tamano_llave)
    return [mensaje[i]^llave[i] for i in range(tamano_llave)]

In [8]:
m = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
c = CifradoFlujo(m)
print(c)
m2 = CifradoFlujo(c)
print(m2)

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### Ejercicio 5
<b>Dada una sucesión de bits periódica, determina la complejidad lineal de dicha sucesión, y el
polinomio de conexión que la genera. Para esto, usa el algoritmo de Berlekamp-Massey.
Haz ejemplos con sumas y productos de secuencias para ver qué ocurre con la complejidad lineal.</b>

In [9]:
def sumaPoli (P, Q):
    if len(P)<=len(Q):
        for i in range (0, len(P)):
            Q[i] = (P[i] ^ Q[i])
        return Q
    else:
        for i in range (0, len(Q)):
            P[i] =  (P[i] ^ Q[i])
        return P


def multiPoli (P, Q):
    if len(P)<=len(Q):
        for i in range (0, len(P)):
            Q[i] = (P[i] * Q[i])
        return Q
    else:
        for i in range (0, len(Q)):
            P[i] =  (P[i] * Q[i])
        return P

    
    
    
def BerlekampMassey(s):
    n = len(s)
    k = -1
    for i in range (n):
        if s[i] == 1:
            k = i
            break
    f = [1] + [0]*k + [1] + [0]*(n-2-k+1)
    g = [1] + [0]*(n)
    l = k+1
    a = k
    b = 0
    r = k+1
    while (r < n):
        d = (sum([f[i]*s[i+r-l] for i in range (0,l+1)]))%2
        
        if (d == 0):
            b = b + 1
        if (d == 1):
            if (2*l > r):
                #[f[i] = f[i]+g[i+(b-a)] for i in range (0,l)]
                for i in range (0,l+1):
                    f[i] = (f[i]^g[i+(b-a)])
                b = b+1
            if (2*l <= r):
                aux = f.copy()
                #[f[i] = aux[i+(a-b)]+g[i] for i in range (0,r+l-1)]
                for i in range (0,r+l-1+1):
                    if(i >= n):
                        i = n-1
                    f[i] = (aux[i+(a-b)]^g[i])
                
                l = r-l+1
                g = aux.copy()
                a = b
                b = r-l+1
        r = r + 1
    for i in range (0, len(f)):
        if (f[-1] == 1):
            f.pop()
            break
        else:
            f.pop()
    return [l, f]


In [10]:
print("El formato del polinomio de salida es s0*x^L+...+sL-1*x. El termino independiente se omite y es igual a 1")


secuencia = LFSR_Stream([1,4],[1,0,1,1], 14)
print("Secuencia generada por 1+x+x^4: ", secuencia)
print("Complejidad y polinomio generador de la secuencia: ",BerlekampMassey(secuencia))

secuencia2 = LFSR_Stream([2,5],[0,1,1,1,0], 14)
print("Secuencia generada por 1+x^2+x^5: ", secuencia2)
print("Complejidad y polinomio generador de la secuencia: ", BerlekampMassey(secuencia2))

secuencia3 = LFSR_Stream([1,7],[0,1,0,0,1,1,1], 14)
print("Secuencia generada por 1+x+x^7: ", secuencia3)
print("Complejidad y polinomio generador de la secuencia: ", BerlekampMassey(secuencia3))


#Comprobamos ahora la complejidad del la suma
print("Vamos a comprobar ahora como afecta sumar y multiplicar secuencias a la complejidad")
suma = sumaPoli(secuencia.copy(), secuencia2.copy())
print("Complejidad y polinomio generador de la suma1: ", BerlekampMassey(suma))
suma = sumaPoli(secuencia.copy(), secuencia3.copy())
print("Complejidad y polinomio generador de la suma2: ", BerlekampMassey(suma))
suma = sumaPoli(secuencia2.copy(), secuencia3.copy())
print("Complejidad y polinomio generador de la suma2: ", BerlekampMassey(suma))

mul = multiPoli(secuencia.copy(), secuencia2.copy())
print("Complejidad y polinomio generador de la multi1: ", BerlekampMassey(mul))
mul = multiPoli(secuencia2.copy(), secuencia3.copy())
print("Complejidad y polinomio generador de la multi2: ", BerlekampMassey(mul))
mul = multiPoli(secuencia.copy(), secuencia3.copy())
print("Complejidad y polinomio generador de la multi3: ", BerlekampMassey(mul))

El formato del polinomio de salida es s0*x^L+...+sL-1*x. El termino independiente se omite y es igual a 1
Secuencia generada por 1+x+x^4:  [1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1]
Complejidad y polinomio generador de la secuencia:  [4, [1, 0, 0, 1]]
Secuencia generada por 1+x^2+x^5:  [0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1]
Complejidad y polinomio generador de la secuencia:  [5, [1, 0, 0, 1, 0]]
Secuencia generada por 1+x+x^7:  [0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1]
Complejidad y polinomio generador de la secuencia:  [7, [1, 0, 0, 0, 0, 0, 1]]
Vamos a comprobar ahora como afecta sumar y multiplicar secuencias a la complejidad
Complejidad y polinomio generador de la suma1:  [6, [0, 0, 0, 0, 0, 0]]
Complejidad y polinomio generador de la suma2:  [8, [1, 1, 1, 1, 0, 1, 0, 1]]
Complejidad y polinomio generador de la suma2:  [8, [0, 1, 0, 1, 1, 0, 0, 0]]
Complejidad y polinomio generador de la multi1:  [5, [0, 1, 0, 0, 1]]
Complejidad y polinomio generador de la multi2:  [8, [0, 1, 